# TIPS

CONVERT(Slot_begin, integer)

( assets.`CONTAINER` LIKE '%uest' OR
   assets.`CONTAINER` = 'SUN zone' OR
   assets.`CONTAINER` = 'HP vpar' )

# Creation / Alteration de table

CREATE TABLE assets_filtree_stockage_réseau
AS 
SELECT *
FROM `assets_filtree`
WHERE YPE IN ('networkcomponents','storage')

Ajout des colonnes pour les coordonnees d'equipements
ALTER TABLE assets_filtree
ADD COLUMN STATUS varchar(255),
ADD COLUMN emplacement varchar(255),
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

# Compteur

SELECT count(*) count, TYPE
from `IT_Equipment`
where STATUS is null
GROUP BY TYPE

SELECT count(*) count, YPE
from `assets_filtree_stockage_réseau`
where STATUS is null
GROUP BY YPE

# Query

In [ ]:
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID
from `IT_Equipment` it
where it.status is null and it.type in ('Stockage', 'Routeurs')

In [ ]:
FOCUS sur un asset null
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID
from `IT_Equipment` it
where SHORT_NAME like '%RCT2%' OR nom like '%RCT2%'

### doublon

In [ ]:
FOCUS sur un asset null
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID
from `IT_Equipment` it
where nom like '%RCT2MF01%'


### N'existe pas

In [ ]:
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID
from `IT_Equipment` it
where it.status is null

In [ ]:
SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE
from `assets_filtree_stockage_réseau` aa
where aa.status is null and NOM_COURT = 'RCT2ST14'

## networkcomponents TYPE

# Liste des it_equipements sans status

In [ ]:
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID, Nom_de_la_salle, Nom_de_baie, Slot_U, Hauteur_U
from `IT_Equipment` it
where it.status is null and it.type in ('Stockage', 'Routeurs')

-- equivalent assets
SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE
from `assets_filtree` aa

### Modele : DS224C

Plusieurs fois le même model avec un seul match. 

select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID, Nom_de_la_salle, Nom_de_baie, Slot_U, Hauteur_U
from `IT_Equipment` it
where modele = 'DS224C'

-- assets
SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE
from `assets_filtree` aa
where model like '%DS224C%' and status is null

# Update

## Match sur les coordonnees

In [ ]:
UPDATE `assets_filtree` aa
INNER JOIN 
	`IT_Equipment` it 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
AND
	it.Hauteur_U = CONVERT(aa.Slot_End, integer) or it.Hauteur_U = CONVERT(aa.Slot_End, integer) - CONVERT(aa.Slot_Begin, integer)

SET aa.status = 'OK-POSITION',
	it.status = 'OK-POSITION'

WHERE aa.status is null and it.status is null